# Data Scraping Code

Instructions to run code:
1. Run all the cells one by one.
2. Change the path in the last cell to save the file. 

In [4]:
from urllib.request import urlopen
from bs4 import BeautifulSoup as BS
import pandas as pd
import numpy as np
import time
import requests
import re


In [5]:
link_group = []
city_list = []
pages = []
company_list = []
job_list = []
salary_list = []
state_list =[]
search_links = ['https://www.indeed.com/jobs?q=%22data+scientist%22+%2420%2C000%2B&l=', 'https://www.indeed.com/jobs?q=%22quantitative+research%22+%2420%2C000%2B&l=', 'https://www.indeed.com/jobs?q=%22data+engineer%22+%2420%2C000%2B&l=',
                'https://www.indeed.com/jobs?q=%22business+intelligence%22+%2420%2C000%2B&l=', 'https://www.indeed.com/jobs?q=%22business+analyst%22+%2420%2C000%2B&l=','https://www.indeed.com/jobs?q=%22data+consultant%22+%2450%2C000&l=']

#find links of the next 5 search pages
for link in search_links:
    website = requests.get(link)
    website_object = BS(website.text, 'html.parser')
    for div in website_object.find_all('div', attrs = {'class':'pagination'}):
        for a in div.find_all('a', attrs = {'onmousedown': 'addPPUrlParam && addPPUrlParam(this);'}):
            pages.append(a['href'])

new_pages = ['https://www.indeed.com{0}'.format(i) for i in pages]

#find individual links in each search page
for link in new_pages:
    search_page = requests.get(link)
    time.sleep(0.1)
    search_page_object = BS(search_page.text, 'html.parser')
    for posting in search_page_object.find_all('div', attrs={'class':'title'}):
        for a in posting.find_all('a', attrs = {'data-tn-element': 'jobTitle'}):
            link_group.append(a['href'])  #get individual posting's link to full page 
    for posting in search_page_object.find_all('div', attrs = {'class': 'recJobLoc'}):
        temp1=posting['data-rc-loc'].split(',')
        city_list.append(temp1[0])
        state_list.append(temp1[len(temp1)-1]) #get location
    for posting in search_page_object.find_all('div', attrs = {'class': 'row'}):
        for a in posting.find_all('a', attrs = {'data-tn-element': 'jobTitle'}):
            job_list.append(a['title']) #get job title
        for span in posting.find_all('span', attrs = {'class':'company'}):
            company_list.append(span.get_text().replace('\n', '')) #get company

   
new_links = ['https://www.indeed.com{0}'.format(i) for i in link_group]


In [6]:
def search_function(page_object): 
    keywords=['bash','python','java','c++','ruby','perl','javascript','scala','matlab'
                          'php','excel','data mining','spss','bigdata',
                          'd3.js','sas','d3','sps','mahout','cobol'
                          'spotfire','scikit.learn','splunk','h2o','hadoop','mapreduce','spark','pig',
                          'hive','shark','sql','nosql','hbase','cassandra','mongodb','mysql','mssql',
                          'postgresql','bigquery','aws','machine learning','deep learning','visualization',
                          'artificial intelligence','azure','tableau', 'alteryx', 'postgres',
                          'sqoop','yarn','c#','etl','unix','linux','shell']
    
    education_list = ['Master','Bachelor','BE','MS','Phd','Doctorate','BA','BS']
    experience_data_list = ['1 year','1+ years','1 or more years','2 years','2+ years','2 or more years','3 years',
                     '3+ years','3 or more years','4 years','4+ years','4 or more years','5 years','5+ years','5 or more years']
    keyword = ''
    experience_temp = []
    temp_exp = []
    industry = []
    degree = ''
    job_description = ''
    salary = ''
    
    for div in page_object.find_all(name = 'div', attrs = {'id':'jobDescriptionText'}):
        job_description = div.get_text().replace('\n', ' ')
        
    job_description.lower()
    
    for i in experience_data_list:
         if(job_description.find(i, 0) > 0):
                experience_temp.append(i)
                
    for m in experience_temp:
        if len(m.split(" ")[0].split("+"))==2:
            temp_exp.append(int(m.split(" ")[0].split("+")[0]))
        else:
            temp_exp.append(int(m.split(" ")[0]))    
    max_exp=0
    if len(temp_exp)==0:
        max_exp=0
    else:
        max_exp=max(temp_exp)
    keyword=''    
    for j in keywords:
         if(job_description.find(j, 0) > 0):
                if keyword=='':
                    keyword=j
                else:
                    keyword=keyword+','+j
    
    for posting in page_object.find_all('div', attrs = {'class': 'jobsearch-JobMetadataHeader-item'}):
        for span in posting.find_all('span', attrs = {'class':'icl-u-xs-mr--xs'}):
            salary = span.get_text().replace('\n', '')
    avg_salary=AvgerageSalary(salary)
            
    for l in education_list:
        if(job_description.find(l,0) > 0):
            degree = l
                
    return {'job_description':job_description,'keyword':keyword,'Salary':avg_salary,'Education':degree,'Experience':max_exp} 


In [7]:
def AvgerageSalary(Salary):
    import math
    AvgSalary=0
    MaxSalary=0
    MinSalary=0

    if Salary=='':
        AvgSalary=0
    else:
        if str(Salary).split(" ")[len(str(Salary).split(" "))-1]=='year':
                MinSalary=float((str(Salary).split(" ")[0].split("$")[1]).replace(",", ''))
                if len(str(Salary).split(" "))==3:
                    MaxSalary=float((str(Salary).split(" ")[0].split("$")[1]).replace(",", ''))
                else:
                    temp1=(str(Salary).split(" ")[2].split("$")[1]).replace(",", '')
                    MaxSalary=float(temp1)  
        elif str(Salary).split(" ")[len(str(Salary).split(" "))-1]=='month': 
                temp=(str(Salary).split(" ")[0].split("$")[1]).replace(",", '')
                MinSalary=float(temp)*12
                temp=(str(Salary).split(" ")[2].split("$")[1]).replace(",", '')
                MaxSalary=float(temp)*12
        elif str(Salary).split(" ")[len(str(Salary).split(" "))-1]=='hour':
                temp1=str(Salary).split(" ")[0].split("$")[1]
                MinSalary=float(temp1)*8*261
                if len(str(Salary).split(" "))==3:
                    MaxSalary=float(temp1)*8*261
                else:
                    temp1=(str(Salary).split(" ")[2].split("$")[1])
                    MaxSalary=float(temp1)*8*261
                    
    AvgSalary=(MinSalary+MaxSalary)/2
    return AvgSalary

In [8]:
i = 0
description_list = []
skill_list=[]
salary_list=[]
education_list=[]
experience_list=[]

while i < len(new_links):
    URL = new_links[i]
    page = requests.get(URL)
    page_object = BS(page.text, 'html.parser')
    result=search_function(page_object) 
    description_list.append(result['job_description'])
    skill_list.append(result['keyword'])
    salary_list.append(result['Salary'])
    education_list.append(result['Education'])
    experience_list.append(result['Experience'])
    i += 1

In [9]:
data = pd.DataFrame(list(zip(job_list, company_list, city_list,state_list, description_list, skill_list, salary_list, education_list,experience_list)), columns = ['Job Title', 'Company', 'City','State','Job Description','Skill Set','Salary', 'Education','Experience'])
#replacing values in Education column as not mentioned
data.Education.replace('', 'Not Mentioned', inplace=True)
data

,Job Title,Company,City,State,Job Description,Skill Set,Salary,Education,Experience
0,Junior Data Scientist,Urner Barry,Toms River,NJ,JOB SUMMARY: The Data Scientist is part of the...,,0.00,Not Mentioned,1
1,Data Scientist,"SubjectWell, Inc",Austin,TX,About SubjectWellSubjectWell helps patients wi...,visualization,140000.00,Not Mentioned,0
2,Staff Data Scientist,Valassis Digital,Morrisville,NC,Staff Data Scientist – Machine Learning & Real...,,0.00,Not Mentioned,5
3,Principal Data Scientist,TARGET,Sunnyvale,CA,Description: JOIN US AS A PRINCIPAL DATA SCIEN...,"data mining,machine learning",0.00,Not Mentioned,0
4,Data Scientist,Verizon,Ashburn,VA,What you’ll be doing... Be a part of the team ...,"excel,data mining,machine learning",0.00,MS,0
5,Undergraduate Internship/Co-op Program - Data ...,Central Intelligence Agency,Washington,DC,As a Data Scientist Undergraduate Intern for t...,,50487.84,Bachelor,0
6,Data Scientist,"K2 Insurance Services, LLC",San Diego,CA,"K2 Insurance Services, LLC is seeking a full-t...",data mining,0.00,BS,5
7,Data Scientist,Albertsons Companies,Pleasanton,CA,Albertsons-Safeway Company is one of the large...,"excel,machine learning,artificial intelligence",0.00,Not Mentioned,2
8,Data Scientist Intern,TWO NIL,Playa Vista,CA,TWO NIL is a growth consultancy that uses a ho...,"machine learning,visualization",0.00,Bachelor,0
9,"Intern, Data Scientist",Sephora,San Francisco,CA,Job ID: 168216 Location Name: CA-FSC SF Off (0...,,0.00,Bachelor,0


In [15]:
data=data[~data[['Job Title','Company','City','Job Description']].apply(frozenset, axis=1).duplicated()]
export_excel = data.to_excel(r'C:\Users\Vamsi\Desktop\Reshma\SEM 2\BAN612-DataAnalytics\Project\612_dataframe.xlsx', index = None, header = True)